In [14]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor

In [57]:
df = pd.read_csv('garments_worker_productivity.csv')
df = df.drop('date', axis=1)
df.head

<bound method NDFrame.head of        quarter  department        day  team  targeted_productivity    smv  \
0     Quarter1      sweing   Thursday     8                   0.80  26.16   
1     Quarter1  finishing    Thursday     1                   0.75   3.94   
2     Quarter1      sweing   Thursday    11                   0.80  11.41   
3     Quarter1      sweing   Thursday    12                   0.80  11.41   
4     Quarter1      sweing   Thursday     6                   0.80  25.90   
...        ...         ...        ...   ...                    ...    ...   
1192  Quarter2   finishing  Wednesday    10                   0.75   2.90   
1193  Quarter2   finishing  Wednesday     8                   0.70   3.90   
1194  Quarter2   finishing  Wednesday     7                   0.65   3.90   
1195  Quarter2   finishing  Wednesday     9                   0.75   2.90   
1196  Quarter2   finishing  Wednesday     6                   0.70   2.90   

         wip  over_time  incentive  idle_time

In [58]:
df['department'].replace('finishing ', 'finishing', inplace=True)
df

C:\Users\99ani\AppData\Local\Temp\ipykernel_19936\4277056618.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['department'].replace('finishing ', 'finishing', inplace=True)


,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [65]:
label_encoder = LabelEncoder()

# Encode the categorical 'feature_2'
df['quarter'] = label_encoder.fit_transform(df['quarter'])
df['department'] = label_encoder.fit_transform(df['department'])
df['day'] = label_encoder.fit_transform(df['day'])
df.fillna(df.median(), inplace=True)

df

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,0,1,3,8,0.80,26.16,1108.000000,7080,98,0.0,0,0,59.0,0.940725
1,0,0,3,1,0.75,3.94,1190.465991,960,0,0.0,0,0,8.0,0.886500
2,0,1,3,11,0.80,11.41,968.000000,3660,50,0.0,0,0,30.5,0.800570
3,0,1,3,12,0.80,11.41,968.000000,3660,50,0.0,0,0,30.5,0.800570
4,0,1,3,6,0.80,25.90,1170.000000,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,1,0,5,10,0.75,2.90,1190.465991,960,0,0.0,0,0,8.0,0.628333
1193,1,0,5,8,0.70,3.90,1190.465991,960,0,0.0,0,0,8.0,0.625625
1194,1,0,5,7,0.65,3.90,1190.465991,960,0,0.0,0,0,8.0,0.625625
1195,1,0,5,9,0.75,2.90,1190.465991,1800,0,0.0,0,0,15.0,0.505889


In [66]:
x = df.iloc[:13, :-1]
y = df.iloc[:13, -1]  # Last column

In [67]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

regressor = LinearRegression()

# Train the model
regressor.fit(x, y)

# Predict on the training set itself since we don't have a separate test set
y_pred = regressor.predict(x)

# Calculate mean squared error
mse = mean_squared_error(y, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R^2 score
r2 = r2_score(y, y_pred)
print(f"R^2 Score: {r2}")


Mean Squared Error: 0.0007319568884750345
R^2 Score: 0.8269930244574125
